In [78]:
from PIL import Image
import numpy as np, PIL, pandas as pd, json, re, pickle, os
from hashlib import sha1 as hash_fn

from time import time, localtime, asctime

from ipywidgets import FloatProgress
from IPython.display import Markdown

In [79]:
hash_fn("test".encode("UTF-8")).hexdigest()

'a94a8fe5ccb19ba61c4c0873d391e987982fbbd3'

In [80]:
start = 20
end = 40

step_size=20

chunk_file_suffix = [i for i in range(start, end, step_size)]

data_dir = "data/intermediate/"
chunking = "chunking/"
hashes = "hashes/"

base_mnist_path = "../mnist/"
by_field_dir = "by_field/"

In [ ]:
def write_hash_pickle(img_hash_dict, file_count):
    """
        Writes a pickle of the passed obj, with filename specifying the hash function, the timestamp and the passed file count
        dict: to serialize
        file_count: shows how many files are processed at a glance (make resuming easier)
        
        returns the filename the pickle is written to
    """
    #generate a unique filename component
    timestamp = asctime(localtime(time())).replace(" ", "_").replace(":", ".")

    #create the filename, including the hash algo
    pickle_filename = "by_field_{}_{}_{}.pickle".format(hash_fn.__name__, timestamp, str(file_count))
    pickle_path = data_dir+hashes+pickle_filename
    
    #open the file
    img_hashes = open(pickle_path, "wb")
    
    #dump the hash dictionary
    pickle.dump(img_hash_dict, img_hashes)

    #close the file
    img_hashes.close()

    return pickle_path

In [ ]:
def read_hash_pickle(filename):
    """
        Read a pickled img hash dictionary, requires filename
        returns image hash based dictionary
    """
    file = open(filename, "rb")
    img_hash_dict = pickle.loads(file.read())
    file.close()
    return img_hash_dict

In [ ]:
#create dict by hash values
hash_img_dict = {}

fp = FloatProgress(min=0, max=100)

files_per_pickle_write = 1000

file_count = 0

display(fp)
for (root, dirs, files) in os.walk(base_mnist_path+by_field_dir):
    clean_path = root.replace("\\", "/")
    
    if len(files) > 0:
        
        for filename in files:
            fp.value = (fp.value+1)%100
            file_path = clean_path+"/"+filename
            filename_pieces = re.split("([0-9]{2})_", filename)
            this_hasher = hash_fn()
            with open(file_path, "rb") as im:
                data = im.read()
                if data is None:
                    display(Markdown("# Error reading file"))
                    
                this_hasher.update(data)
                
            #explicitly close the file for better resource management
            im.close()
            
            digest = this_hasher.hexdigest()
            
            if digest in hash_img_dict.keys():
                existing_file_with_hash = hash_img_dict[digest]["file_path"]
                markdown_msg = "# Collision!\n ## Existing file: {} New File: {}".format(existing_file_with_hash, file_path)
                display(Markdown(markdown_msg))
            
            #display(filename_pieces, digest)
            
            if len(filename_pieces) < 2:
                continue
                
            code = filename_pieces[1]
            #display(code)
            hash_img_dict[digest] = {"file_path": file_path, "char_label": code}
            
            file_count += 1
            
            if file_count % files_per_pickle_write == 0:
                write_hash_pickle(hash_img_dict, file_count)

FloatProgress(value=0.0)

In [ ]:
write_hash_pickle(hash_img_dict, 0)

In [ ]:
reloaded = read_hash_pickle("data/intermediate/hashes/by_field_openssl_sha1_Tue_Mar__5_18.19.19_2019_0.pickle")

In [ ]:
reloaded == hash_img_dict

In [ ]:
by_writer_dir = "by_write/"

this_hash = hash_fn()

with open(base_mnist_path+by_writer_dir+"hsf_0/f0000_14/c0000_14/c0000_14_00000.png", "rb") as test:    
    data = test.read()
    this_hash.update(data)
    
digest = this.hexdigest()
    
if digest in hash_img_dict.keys():
    hash_img_dict[digest]